In [1]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import xGPR
from xGPR import xGPRegression as xGPReg, FastConv1d
from xGPR.data_handling.dataset_builder import build_regression_dataset
import antpack
from sklearn.metrics import r2_score
import optuna
from xgboost import XGBRegressor

if "notebooks" in os.getcwd():
    start_dir = os.path.join(os.getcwd(), "..")

os.chdir(start_dir)

from multi_target_modeling_src.data_encoding.seq_encoder_functions import OneHotEncoder, PFAStandardEncoder, PChemPropEncoder

/ssd1/Documents/Wang_Lab_Projects/COVID/venv_supplementary/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.chdir(os.path.join(start_dir, "supplementary_experiment_data_and_results", "mit-ll-AlphaSeq_Antibody_Dataset-a8f64a9",
                     "antibody_dataset_1", "MITLL_AAlphaBio_Ab_Binding_dataset.csv"))
raw_data = pd.read_csv("MITLL_AAlphaBio_Ab_Binding_dataset.csv")

In [3]:
raw_data.columns

Index(['POI', 'Sequence', 'Target', 'Assay', 'Replicate', 'Pred_affinity',
       'HC', 'LC', 'CDRH1', 'CDRH2', 'CDRH3', 'CDRL1', 'CDRL2', 'CDRL3'],
      dtype='object')

In [4]:
# For this particular benchmark exercise, we are only interested in binding to the target.
raw_data = raw_data[raw_data["Target"]=="MIT_Target"]

# As in the literature, if there are multiple values for one sequence, average over them.
agg_fun = {col:"first" for col in raw_data.columns.tolist()}
agg_fun["Pred_affinity"] = "mean"
raw_data = raw_data.groupby('Sequence').aggregate(agg_fun)
print(raw_data.shape)

raw_data = raw_data.loc[(raw_data['Pred_affinity'] < 3) | (raw_data['Pred_affinity'] >= 4)].copy()
print(raw_data.shape)

(104968, 14)
(51590, 14)


In [5]:
from antpack import SingleChainAnnotator as SCA
sca_light = SCA(chains=["K", "L"])
sca_heavy = SCA(chains=["H"])

cdr_list = ["CDRH1", "CDRH2", "CDRH3", "CDRL1", "CDRL2", "CDRL3"]
cdr_numbering = [[] for cdr in cdr_list]

for i, seq in enumerate(raw_data["Sequence"].tolist()):
    h_res = sca_heavy.analyze_seq(seq)[0]
    for j, cdr in enumerate(cdr_list[:3]):
        cdr_seq = raw_data[cdr].values[i]
        begin = seq.find(cdr_seq)
        end = begin + len(cdr_seq)
        cdr_numbering[j].append(h_res[begin:end])

    l_res = sca_light.analyze_seq(seq)[0]
    for j, cdr in enumerate(cdr_list[3:]):
        cdr_seq = raw_data[cdr].values[i]
        begin = seq.find(cdr_seq)
        end = begin + len(cdr_seq)
        cdr_numbering[j+3].append(l_res[begin:end])

In [6]:
expected_positions = {}

for i, cdr in enumerate(cdr_list):
    unique_numbers = []
    for c in cdr_numbering[i]:
        unique_numbers += c

    expected_positions[cdr] = np.sort(np.unique(unique_numbers))

In [7]:
full_length = sum([len(expected_positions[k]) for k in expected_positions.keys()])

net_length = 0
encoded_seqs = [["-" for k in range(full_length)] for s in raw_data["Sequence"].tolist()]

for j, cdr in enumerate(cdr_list):
    seqs = raw_data[cdr].tolist()
    seq_idx = {c:(i+net_length) for i, c in enumerate(expected_positions[cdr])}

    for seq, encoded_seq,numbering in zip(seqs, encoded_seqs, cdr_numbering[j]):
        for letter, pos in zip(seq, numbering):
            encoded_seq[seq_idx[pos]] = letter
    
    net_length += len(seq_idx)

encoded_seqs = ["".join(s) for s in encoded_seqs]

In [8]:
ohencoder = OneHotEncoder()
pchemencoder = PChemPropEncoder()
pfaencoder = PFAStandardEncoder()

In [21]:
fcdata = pchemencoder.encode_variable_length(encoded_seqs)
fcdata = fcdata.reshape((fcdata.shape[0], 78*3))
rng = np.random.default_rng(123)
idx = rng.permutation(fcdata.shape[0])
trainidx, validx, testidx = idx[:41787], idx[41787:(41787+4644)], idx[(41787+4644):]

yvalues = raw_data["Pred_affinity"].values.astype(np.float64)
trainx, trainy = fcdata[trainidx,...], yvalues[trainidx]
valx, valy = fcdata[validx,...], yvalues[validx]
testx, testy = fcdata[testidx,...], yvalues[testidx]

In [26]:
import sklearn
from sklearn.model_selection import KFold
def objective(trial):
    kf = KFold(n_splits=5)

    kf.get_n_splits(trainx)

    params = {
            'max_depth': trial.suggest_int('max_depth', 2, 6),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 50, 500),
            'min_child_weight': trial.suggest_int('min_child_weight', 0.1, 10),
            'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0, log=True),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.05, 1.0, log=True),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
            'eval_metric': 'mlogloss',
    }

    optuna_model = XGBRegressor(**params)
    results = []
    for i, (train_index, test_index) in enumerate(kf.split(trainx)):
        subtrainx, subvalidx = trainx[train_index,...], trainx[test_index,...]
        subtrainy, subvalidy = trainy[train_index], trainy[test_index]
        optuna_model.fit(subtrainx, subtrainy)
        y_pred = optuna_model.predict(subvalidx)
        score = np.mean(np.abs(subvalidy - y_pred))
        results.append(score)

    return np.mean(results)

In [27]:
sampler = optuna.samplers.TPESampler(seed=123)
study = optuna.create_study(sampler=sampler, direction='minimize')
study.optimize(objective, n_trials=100)

[I 2024-05-30 08:43:34,390] A new study created in memory with name: no-name-98579afe-4d2c-47d5-8e03-db31c887f86e
[I 2024-05-30 08:43:35,624] Trial 0 finished with value: 1.0606805726586264 and parameters: {'max_depth': 5, 'learning_rate': 0.030629657963841296, 'n_estimators': 152, 'min_child_weight': 6, 'gamma': 0.005698384608345678, 'subsample': 0.6704057649058658, 'colsample_bytree': 0.9440035882733008, 'reg_alpha': 0.003010494989157965, 'reg_lambda': 7.03809641382707e-05}. Best is trial 0 with value: 1.0606805726586264.
[I 2024-05-30 08:43:37,316] Trial 1 finished with value: 1.0392321543486385 and parameters: {'max_depth': 3, 'learning_rate': 0.03828680757584917, 'n_estimators': 378, 'min_child_weight': 4, 'gamma': 3.0020863018484457e-08, 'subsample': 0.6588601863947399, 'colsample_bytree': 0.456167314561352, 'reg_alpha': 2.883592210117951e-07, 'reg_lambda': 2.53287670148807e-07}. Best is trial 1 with value: 1.0392321543486385.
[I 2024-05-30 08:43:39,066] Trial 2 finished with val

In [28]:
trial = study.best_trial
params = trial.params
xgboost_model = XGBRegressor(**params)
xgboost_model.fit(trainx, trainy)

preds = xgboost_model.predict(testx)

In [55]:
fcdata.shape

(51590, 78, 21)

In [58]:
#cdrseqs = [raw_data[col].tolist() for col in ["CDRH1", "CDRH2", "CDRH3", "CDRL1", "CDRL2", "CDRL3"]]


#oh_encodings = [pfaencoder.encode_variable_length(cdrset) for cdrset in cdrseqs]
fcdata = ohencoder.encode(encoded_seqs)
fcdata = fcdata.reshape((fcdata.shape[0], fcdata.shape[1] * fcdata.shape[2])).astype(np.float32)
#slengths = [np.array([len(s) for s in cdrset]).astype(np.int32) for cdrset in cdrseqs]
#fconv = FastConv1d(seq_width=21, device="gpu", random_seed=123, conv_width=5, num_features=1024)

#fcdata = [fconv.predict(x, s, chunk_size=1000) for (x,s) in zip(oh_encodings, slengths)]
#fcdata = np.hstack(fcdata).astype(np.float32)

In [59]:
rng = np.random.default_rng(123)
idx = rng.permutation(fcdata.shape[0])
trainidx, validx, testidx = idx[:41787], idx[41787:(41787+4644)], idx[(41787+4644):]

yvalues = raw_data["Pred_affinity"].values.astype(np.float64)
trainx, trainy = fcdata[trainidx,...], yvalues[trainidx]
valx, valy = fcdata[validx,...], yvalues[validx]
testx, testy = fcdata[testidx,...], yvalues[testidx]

regdata = build_regression_dataset(trainx, trainy, chunk_size=1000)

In [60]:
xg = xGPReg(num_rffs=2048, kernel_choice="RBF", device="gpu", variance_rffs=12)
xg.tune_hyperparams_crude(regdata)

Grid point 0 acquired.
Grid point 1 acquired.
Grid point 2 acquired.
Grid point 3 acquired.
Grid point 4 acquired.
Grid point 5 acquired.
Grid point 6 acquired.
Grid point 7 acquired.
Grid point 8 acquired.
Grid point 9 acquired.
New hparams: [-2.5040047]
Additional acquisition 10.
New hparams: [-2.0720023]
Additional acquisition 11.
New hparams: [-2.9752086]
Additional acquisition 12.
New hparams: [-2.4909924]
Best score achieved: 47557.343
Best hyperparams: [-1.4621671 -2.4909924]


(array([-1.4621671, -2.4909924]), 13, 47557.343)

In [61]:
xg.num_rffs = 8192
xg.fit(regdata, mode="cg")

starting fitting
Chunk 0 complete.
Chunk 10 complete.
Chunk 20 complete.
Chunk 30 complete.
Chunk 40 complete.
Using rank: 512
Chunk 0 complete.
Chunk 10 complete.
Chunk 20 complete.
Chunk 30 complete.
Chunk 40 complete.
Iteration 0
Iteration 5
Iteration 10
Iteration 15
Iteration 20
Now performing variance calculations...
Fitting complete.


In [62]:
preds = xg.predict(testx)

In [30]:
from scipy.stats import spearmanr
print(spearmanr(testy, preds))

SignificanceResult(statistic=0.6715765658353137, pvalue=0.0)


In [31]:
valcat = testy.copy()

valcat[valcat<=3]=0
valcat[valcat>3]=1

In [32]:
from sklearn.metrics import average_precision_score, roc_auc_score
roc_auc_score(valcat, preds)

0.8762080023815001

In [33]:
average_precision_score(valcat, preds)

0.873566666188171